In [1]:
import os
os.chdir("./..")

In [2]:
# Check Pytorch installation
import torch, torchvision
print("torch version:",torch.__version__, "cuda:",torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print("mmdetection:",mmdet.__version__)

# Check mmcv installation
import mmcv
print("mmcv:",mmcv.__version__)

# Check mmengine installation
import mmengine
print("mmengine:",mmengine.__version__)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
torch version: 2.0.1 cuda: False
mmdetection: 3.3.0
mmcv: 2.0.0rc4
mmengine: 0.10.4


In [3]:
os.listdir()

['demo',
 'tools',
 'docker',
 'pytest.ini',
 'toy_dataset',
 'LICENSE',
 'requirements.txt',
 '.pre-commit-config-zh-cn.yaml',
 '.pre-commit-config.yaml',
 'CITATION.cff',
 '.owners.yml',
 'file_processing',
 'projects',
 'resources',
 'checkpoints',
 'out.png',
 'tests',
 'MANIFEST.in',
 'requirements',
 'docs',
 'README_zh-CN.md',
 '.readthedocs.yml',
 'README.md',
 '__MACOSX',
 'test.png',
 'setup.py',
 'inference_scripts',
 '.gitignore',
 'mmdet.egg-info',
 'configs',
 '.github',
 '.dev_scripts',
 'setup.cfg',
 '.ipynb_checkpoints',
 '.git',
 'model-index.yml',
 '.circleci',
 'dataset-index.yml',
 'mmdet']

## Perform Inference with An MMDetection Detector

In [4]:
import mmcv
import mmengine
from mmdet.apis import init_detector, inference_detector
from mmdet.utils import register_all_modules
# Choose to use a config and initialize the detector
config_file = 'configs/mask_rcnn/mask-rcnn_r50-caffe_fpn_ms-poly-3x_coco.py'
# Setup a checkpoint file to load
checkpoint_file = 'checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth'

# register all modules in mmdet into the registries
register_all_modules()

# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cpu')  # or device='cuda:0'


Loads checkpoint by local backend from path: checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth


In [5]:
from mmengine import Config
cfg = Config.fromfile('./configs/mask_rcnn/mask-rcnn_r50-caffe_fpn_ms-poly-1x_coco.py')

In [6]:

from mmdet.datasets.transforms import (AutoAugment, AutoContrast, Brightness,
                                       Color, Contrast, Equalize, Invert,
                                       Posterize, RandAugment, Rotate,
                                       Sharpness, ShearX, ShearY, Solarize,
                                       SolarizeAdd, TranslateX, TranslateY)

from mmdet.datasets import CocoDataset


In [7]:
data_root = './toy_dataset/balloon'
metainfo = dict(classes=('balloon'), task_name='balloon')
from mmengine import Config
file_name = 'configs/mask2former/mask2former_r50_8xb2-lsj-50e_coco.py'
cfg = Config.fromfile(file_name)

dataset = CocoDataset(
            data_root="toy_dataset/balloon/",
            data_prefix=dict(img='train/'),
            metainfo = metainfo,
            ann_file='balloon_train.json',
            filter_cfg=dict(filter_empty_gt=True, min_size=32),
            pipeline=[],
            serialize_data=False,
            lazy_init=False)

len(dataset)

07/02 15:42:51 - mmengine - WARNING - balloon is not a meta file, simply parsed as meta information
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


61

In [8]:
from imagecorruptions import get_corruption_names
get_corruption_names()

['gaussian_noise',
 'shot_noise',
 'impulse_noise',
 'defocus_blur',
 'glass_blur',
 'motion_blur',
 'zoom_blur',
 'snow',
 'frost',
 'fog',
 'brightness',
 'contrast',
 'elastic_transform',
 'pixelate',
 'jpeg_compression']

In [9]:
cfg.train_dataloader.dataset 

{'type': 'CocoDataset',
 'data_root': 'data/coco/',
 'ann_file': 'annotations/instances_train2017.json',
 'data_prefix': {'img': 'train2017/',
  'seg': 'annotations/panoptic_train2017/'},
 'filter_cfg': {'filter_empty_gt': True, 'min_size': 32},
 'pipeline': [{'type': 'LoadImageFromFile',
   'to_float32': True,
   'backend_args': None},
  {'type': 'LoadAnnotations', 'with_bbox': True, 'with_mask': True},
  {'type': 'RandomFlip', 'prob': 0.5},
  {'type': 'RandomResize',
   'scale': (1024, 1024),
   'ratio_range': (0.1, 2.0),
   'resize_type': 'Resize',
   'keep_ratio': True},
  {'type': 'RandomCrop',
   'crop_size': (1024, 1024),
   'crop_type': 'absolute',
   'recompute_bbox': True,
   'allow_negative_crop': True},
  {'type': 'FilterAnnotations',
   'min_gt_bbox_wh': (1e-05, 1e-05),
   'by_mask': True},
  {'type': 'PackDetInputs'}],
 'backend_args': None}

In [10]:
if False:
    cfg.train_dataloader.dataset = dict(
            type = 'CocoDataset',
            data_prefix=dict(img='train'),
            data_root=data_root,
            ann_file='balloon_train.json',
            metainfo=metainfo,
            filter_cfg=dict(filter_empty_gt=True, min_size=32),
            pipeline= [
                {'type': 'LoadImageFromFile',
                   'to_float32': True,
                   'backend_args': None},
                   {'type': 'LoadAnnotations',
                            'with_bbox': True,
                            'with_mask': True,
                            'poly2mask': True},
                  {'type': 'FilterAnnotations',
                   'min_gt_bbox_wh': (1e-05, 1e-05),
                   'by_mask': True},
                  {'type': 'PackDetInputs'}
            ],
            serialize_data=False,
            lazy_init=False)

    if False:
        {'type': 'RandomElasticDeformation',
                            'min_sigma': 25,
                            'max_sigma': 26,
                            'min_points': 3,
                            'max_points': 4},
        {'type': 'RandomColorFormCutOutAlpha',
                            "n_holes": 9,
                            "cutout_shape": [(128,128)],
                            "fill_min": 0, 
                            "fill_max": 255,
                            "alpha_min":0.2,
                            "alpha_max":0.8
                    },
        {'type': 'PhotoMetricDistortion',
                            "brightness_delta": 0, 
                            "contrast_range": 0, 
                            "saturation_range": 0, 
                            "hue_delta": 0, 
                    },
        {'type': 'Corrupt',
                            "corruption": "jpeg_compression",
                            "severity": 1,
                        }
        
        "Corrupt"
        if False:
            'gaussian_noise',
            'shot_noise',
            'impulse_noise',
            'defocus_blur',
            'motion_blur',
            'zoom_blur',
            'snow',
            'frost',
            'fog',
            'brightness',
            'contrast',
            'elastic_transform',
            'pixelate',
            'jpeg_compression'

        "PhotoMetricDistortion"
        if False:
            "brightness_delta"
            "contrast_range"
            "saturation_range"
            "hue_delta"

    
    
    
    
    
    

img_scales = [
            (384, 384),
            (512, 512),
            (640, 640),
            (768, 768),
    ]

#Corrupt
#PhotoMetricDistortion

cfg.train_dataloader.dataset = dict(
            type = 'CocoDataset',
            data_prefix=dict(img='train'),
            data_root=data_root,
            ann_file='balloon_train.json',
            metainfo=metainfo,
            filter_cfg=dict(filter_empty_gt=True, min_size=32),
            pipeline= [
                {'type': 'LoadImageFromFile',
                       'to_float32': True,
                       'backend_args': None},
                {'type': 'LoadAnnotations',
                    'with_bbox': True,
                    'with_mask': True,
                    'poly2mask': True
                },
                {'type': 'RandomChoiceResize',
                'scales': img_scales,
                'keep_ratio': False
                },
                {"type": "RandomRandAugment",
                 "aug_space": [
                     [
                         {"type": "RandomColorFormCutOut",
                         "n_holes": (0,4),
                         "cutout_shape": [(16,16),(24,24),(32,32),(48,48),(64,64),(72,72)]
                        },
                     ],
                     [
                        {"type": "RandomColorFormCutOutAlpha",
                         "n_holes": (0,4),
                         "cutout_shape": [(16,16),(24,24),(32,32),(48,48),(64,64),(72,72)]
                        },
                     ],
                     [
                         {'type': 'RandomElasticDeformation',
                            'min_sigma': 0,
                            'max_sigma': 26,
                            'min_points': 1,
                            'max_points': 4},
                     ],
                     [
                         {'type': 'Corrupt',
                            "corruption": "gaussian_noise",
                            "severity": 1,
                        }
                     ],
                     [
                         {'type': 'Corrupt',
                            "corruption": "shot_noise",
                            "severity": 1,
                        }
                     ],
                     [
                         {'type': 'Corrupt',
                            "corruption": "impulse_noise",
                            "severity": 1,
                        }
                     ],
                     [
                         {'type': 'Corrupt',
                            "corruption": "defocus_blur",
                            "severity": 1,
                        }
                     ],
                     [
                         {'type': 'Corrupt',
                            "corruption": "motion_blur",
                            "severity": 1,
                        }
                     ],
                     [
                         {'type': 'Corrupt',
                            "corruption": "zoom_blur",
                            "severity": 1,
                        }
                     ],
                     [
                         {'type': 'Corrupt',
                            "corruption": "snow",
                            "severity": 1,
                        }
                     ],
                     [
                         {'type': 'Corrupt',
                            "corruption": "frost",
                            "severity": 1,
                        }
                     ],
                     [
                         {'type': 'Corrupt',
                            "corruption": "fog",
                            "severity": 1,
                        }
                     ],
                     [
                         {'type': 'Corrupt',
                            "corruption": "brightness",
                            "severity": 1,
                        }
                     ],
                     [
                         {'type': 'Corrupt',
                            "corruption": "contrast",
                            "severity": 1,
                        }
                     ],
                     [
                         {'type': 'Corrupt',
                            "corruption": "elastic_transform",
                            "severity": 1,
                        }
                     ],
                     [
                         {'type': 'Corrupt',
                            "corruption": "pixelate",
                            "severity": 1,
                        }
                     ],
                     [
                         {'type': 'Corrupt',
                            "corruption": "jpeg_compression",
                            "severity": 1,
                        }
                     ],
                     [
                         {'type': 'PhotoMetricDistortion',
                            "brightness_delta": 32,
                            "contrast_range": (0.5, 1.5),
                            "saturation_range": (0.5, 1.5),
                            "hue_delta": 18
                        }
                     ],
                     [dict(type='Invert')], 
                     [dict(type='Solarize')],
                     [dict(type='SolarizeAdd')], 
                     [dict(type='Color')],
                     [dict(type='Contrast')], 
                     [dict(type='Brightness')],
                     [dict(type='Sharpness')], 
                     [dict(type='Rotate')],
                     [dict(type='ShearX')],
                     [dict(type='ShearY')], 
                     [dict(type='TranslateX')],
                     [dict(type='TranslateY')]
                 ],
                 "aug_num": 6,
                 
                },
                
            ],
            serialize_data=False,
            lazy_init=False)
                                                  
img_scale = (640,640)
train_pipeline = [
    dict(type="CutMix",
        ratio_range =  (0.5,1.5),
        flip_ratio =  0.5,
        pad_val =  114.0,
        max_iters =  15,
        bbox_clip_border =  True ,
        crop_min= 64,
        crop_max= 256,
        p = 0.5
        ),
    dict(type='PackDetInputs')
]

cfg.train_dataloader.dataset = dict(
    type='MultiImageMixDataset',
    dataset= cfg.train_dataloader.dataset,
    pipeline=train_pipeline
    )

In [11]:
import numpy as np
np.random.uniform()

0.5215763361556962

In [14]:
# Step 1: Import necessary libraries
import mmcv
import matplotlib.pyplot as plt

from mmdet.models.utils import mask2ndarray
from mmdet.registry import DATASETS, VISUALIZERS
from mmdet.structures.bbox import BaseBoxes
from mmengine.registry import init_default_scope
from mmengine.utils import ProgressBar

import os.path as osp

# register all modules in mmdet into the registries
init_default_scope('mmdet')

dataset = DATASETS.build(
    cfg.train_dataloader.dataset
)

visualizer = VISUALIZERS.build(
    dict(
        type='DetLocalVisualizer',
        vis_backends=[dict(type='LocalVisBackend')],
        name='visualizer'
    )
)
visualizer.dataset_meta = dataset.metainfo

progress_bar = ProgressBar(len(dataset))

# item = dataset[5]

for item in dataset:
    print(1)
    img = item['inputs'].permute(1, 2, 0).numpy()
    data_sample = item['data_samples'].numpy()
    gt_instances = data_sample.gt_instances
    img_path = osp.basename(item['data_samples'].img_path)


    img = img[..., [2, 1, 0]]  # bgr to rgb
    gt_bboxes = gt_instances.get('bboxes', None)
    if gt_bboxes is not None and isinstance(gt_bboxes, BaseBoxes):
        gt_instances.bboxes = gt_bboxes.tensor
    gt_masks = gt_instances.get('masks', None)
    if gt_masks is not None:
        masks = mask2ndarray(gt_masks)
        gt_instances.masks = masks.astype(bool)
    data_sample.gt_instances = gt_instances

    visualizer.add_datasample(
            osp.basename(img_path),
            img,
            data_sample,
            draw_pred=True,
            out_file="out.png")

    visualizer.show()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
[                                                  ] 0/61, elapsed: 0s, ETA:1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [ ]:
# Step 1: Import necessary libraries
import mmcv
import matplotlib.pyplot as plt

from mmdet.models.utils import mask2ndarray
from mmdet.registry import DATASETS, VISUALIZERS
from mmdet.structures.bbox import BaseBoxes
from mmengine.registry import init_default_scope
from mmengine.utils import ProgressBar

import os.path as osp

# register all modules in mmdet into the registries
init_default_scope('mmdet')

dataset = DATASETS.build(
    cfg.train_dataloader.dataset
)

visualizer = VISUALIZERS.build(
    dict(
        type='DetLocalVisualizer',
        vis_backends=[dict(type='LocalVisBackend')],
        name='visualizer'
    )
)
visualizer.dataset_meta = dataset.metainfo

progress_bar = ProgressBar(len(dataset))

item = dataset[0]

#for item in dataset:

img = item['inputs'].permute(1, 2, 0).numpy()
data_sample = item['data_samples'].numpy()
gt_instances = data_sample.gt_instances
img_path = osp.basename(item['data_samples'].img_path)


img = img[..., [2, 1, 0]]  # bgr to rgb
gt_bboxes = gt_instances.get('bboxes', None)
if gt_bboxes is not None and isinstance(gt_bboxes, BaseBoxes):
    gt_instances.bboxes = gt_bboxes.tensor
gt_masks = gt_instances.get('masks', None)
if gt_masks is not None:
    masks = mask2ndarray(gt_masks)
    gt_instances.masks = masks.astype(bool)
data_sample.gt_instances = gt_instances

visualizer.add_datasample(
        osp.basename(img_path),
        img,
        data_sample,
        draw_pred=True,
    
        out_file="test.png")

visualizer.show()

In [ ]:
from PIL import Image
Image.fromarray(img)

In [ ]:
import copy

# test RandAugment equipped with Rotate
aug_space = [[
    dict(type='Rotate', prob=1.0, level=10, reversal_prob=0.0)
]]
transform_rand = RandAugment(aug_space=aug_space, aug_num=1)
results_rand = transform_rand(copy.deepcopy(self.results_mask))
transform_rotate = Rotate(prob=1.0, level=10, reversal_prob=0.0)

results_rotated = transform_rotate(copy.deepcopy(self.results_mask))

check_result_same(results_rotated, results_rand, self.check_keys)

# test RandAugment equipped with default augmentation space
transform_rand = RandAugment()
transform_rand(copy.deepcopy(self.results_mask))

### Train a new detector

Finally, lets initialize the dataset and detector, then train a new detector!

In [ ]:
!python tools/train.py {config}

### Understand the log
From the log, we can have a basic understanding on the training process and know how well the detector is trained.

First, since the dataset we are using is small, we loaded a Mask R-CNN model and finetune it for detection. Because the original Mask R-CNN is trained on COCO dataset that contains 80 classes but KITTI Tiny dataset only have 3 classes. Therefore, the last FC layers of the pre-trained Mask R-CNN for classification and regression have different weight shape and are not used. The pre-trained weights of mask prediction layer `mask_head.conv_logits` also does not matches the current model and is not used due to similar reason.

Third, after training, the detector is evaluated by the default COCO-style evaluation. The results show that the detector achieves 79.6 bbox AP and 81.5 mask AP on the val dataset, not bad!

 We can also check the tensorboard to see the curves.

In [ ]:
%pip install tensorboard  -i https://mirrors.ustc.edu.cn/pypi/web/simple

In [ ]:
# load tensorboard in jupyter notebook
%load_ext tensorboard

In [ ]:
# see curves in tensorboard
# if you see <IPython.core.display.HTML object> please run it again
%tensorboard --logdir tutorial_exps/

## Test the Trained Detector

After finetuning the detector, let's visualize the prediction results!

In [ ]:
import mmcv
from mmdet.apis import init_detector, inference_detector
img = mmcv.imread('./ballondatasets/balloon/train/7178882742_f090f3ce56_k.jpg',channel_order='rgb')
checkpoint_file = 'tutorial_exps/epoch_12.pth'
model = init_detector(cfg, checkpoint_file, device='cpu')
new_result = inference_detector(model, img)
print(new_result)

In [ ]:
from mmengine.visualization import Visualizer
# get built visualizer
visualizer_now = Visualizer.get_current_instance()
# the dataset_meta is loaded from the checkpoint and
# then pass to the model in init_detector
visualizer_now.dataset_meta = model.dataset_meta
# show the results
visualizer_now.add_datasample(
    'new_result',
    img,
    data_sample=new_result,
    draw_gt=False,
    wait_time=0,
    out_file=None,
    pred_score_thr=0.5
)
visualizer_now.show()

## What to Do Next?

So far, we have learnt how to test and train Mask R-CNN. To further explore the segmentation task, you could do several other things as shown below:

- Try cascade methods, e.g., [Cascade Mask R-CNN](https://github.com/open-mmlab/mmdetection/tree/master/configs/cascade_rcnn) and [HTC](https://github.com/open-mmlab/mmdetection/tree/master/configs/htc) in [MMDetection model zoo](https://github.com/open-mmlab/mmdetection/blob/master/docs/en/model_zoo.md). They are powerful detectors that are ranked high in many benchmarks, e.g., COCO dataset.
- Try single-stage methods, e.g., [K-Net](https://github.com/ZwwWayne/K-Net) and [Dense-RepPoints](https://github.com/justimyhxu/Dense-RepPoints). These two algorithms are based on MMDetection. Box-free instance segmentation is a new trend in the instance segmentation community.
- Try semantic segmentation. Semantic segmentation is also a popular task with wide applications. You can explore [MMSegmentation](https://github.com/open-mmlab/mmsegmentation/); we also provide a [colab tutorial](https://github.com/open-mmlab/mmsegmentation/blob/master/demo/MMSegmentation_Tutorial.ipynb) for semantic segmentation using MMSegmentation.
